### Against Win Rate

Computes win rate against opponent.

In [1]:
import pandas as pd

games_df = pd.read_csv("../../data/processed/v3.csv")

games_df.sort_values(by=['date'], inplace=True, ignore_index=True)
games_df

,win_name,lose_name,win_color,lose_color,date,win_rank,lose_rank,komi
0,Kato Shin,Sekiyama Riichi,b,w,1941-05-18,7p,6p,0.0
1,Sekiyama Riichi,Kato Shin,b,w,1941-05-27,6p,7p,0.0
2,Kato Shin,Sekiyama Riichi,b,w,1941-06-09,7p,6p,0.0
3,Sekiyama Riichi,Kato Shin,b,w,1941-06-21,6p,7p,0.0
4,Kato Shin,Sekiyama Riichi,b,w,1941-07-01,7p,6p,0.0
...,...,...,...,...,...,...,...,...
101404,Jin Siyoung,Choi Kyeongho,w,b,2022-02-19,8p,5p,6.5
101405,Kang Dongyun,Seol Hyunjun,b,w,2022-02-19,9p,7p,6.5
101406,Kim Jiseok,Shin Jaeweon,w,b,2022-02-19,9p,4p,6.5
101407,Kim Changhoon,Kwak Wonkeun,b,w,2022-02-19,5p,3p,6.5


In [10]:
games_df = games_df.rename(columns={'win_name': 'player', 'lose_name':'opponent'})

In [11]:
from tqdm import tqdm

"""
    Compute every player's cumulative win rates agaisnt opponent
    
    AWR = Against Win Rate
"""


def computeAWR(x):
    n = x["num_games"]
    for i in range(0, x["num_games"] + 1):
        if (n - i) - i == x["AS"]:
            return (n - i) / n if x["reverted"] == 1 else i / n


games_df["AWR"] = 0
against_df = games_df.copy(deep=True)
against_df["match"] = (
    against_df[["player", "opponent"]].apply(sorted, axis=1).astype(str)
)

against_df = against_df.groupby("match")

for name, group in tqdm(against_df):
    group["tolist"] = [
        str(val) for val in group[["player", "opponent"]].values.tolist()
    ]
    group["reverted"] = group.apply(lambda x: 1 if x["tolist"] == name else -1, axis=1)
    group["num_games"] = range(1, len(group) + 1)
    group["AS"] = group["reverted"].rolling(5, min_periods=1).sum()
    group["AWR"] = group.apply(computeAWR, axis=1)

    group = group["AWR"]
    
    group = group.shift()
    games_df.update(group)

games_df

100%|██████████| 43249/43249 [06:16<00:00, 114.90it/s]


,player,opponent,win_color,lose_color,date,win_rank,lose_rank,komi,AWR
0,Kato Shin,Sekiyama Riichi,b,w,1941-05-18,7p,6p,0.0,0.000000
1,Sekiyama Riichi,Kato Shin,b,w,1941-05-27,6p,7p,0.0,1.000000
2,Kato Shin,Sekiyama Riichi,b,w,1941-06-09,7p,6p,0.0,0.500000
3,Sekiyama Riichi,Kato Shin,b,w,1941-06-21,6p,7p,0.0,0.666667
4,Kato Shin,Sekiyama Riichi,b,w,1941-07-01,7p,6p,0.0,0.500000
...,...,...,...,...,...,...,...,...,...
101404,Jin Siyoung,Choi Kyeongho,w,b,2022-02-19,8p,5p,6.5,0.000000
101405,Kang Dongyun,Seol Hyunjun,b,w,2022-02-19,9p,7p,6.5,0.800000
101406,Kim Jiseok,Shin Jaeweon,w,b,2022-02-19,9p,4p,6.5,0.000000
101407,Kim Changhoon,Kwak Wonkeun,b,w,2022-02-19,5p,3p,6.5,0.000000
